### CPSC 672 Network Project

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import random
from random import sample
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import ast
from scipy import spatial
import csv
from tqdm.notebook import tqdm
%matplotlib inline

### Data

The following few cells read in our data from a excel spreadsheet and organize them into a few arrays:
- stock_symbols contains all the actual stock names
- dates contains all the column headings (which are dates) from Feb 2010 to Dec 2021

In [64]:
df_total = pd.read_excel('data/all_data.xlsx', sheet_name='percentChange')
df_subset = pd.read_excel('data/subset_data.xlsx',sheet_name='percentChange')

In [65]:
stock_symbols = []
for stock in df_total['Stock']:
    stock_symbols.append(stock)

dates = df_total.columns.values
dates = dates[2:]

### Calculating links

To get the links between any two stocks, we use cosine similarity to determine how correlated two stocks are. The formula for cosine similarity is as follows:
$$ 
x = {1 - \frac{u \cdot v}{||u|| * ||v||}  }
$$
                            
the getSimilarities() functions do just that by comparing every stock, with every other stock, for any input date (column). Note one of the functions calculates this over a 12 month period (Yearly) whereas the other function uses the total dataset.

In [100]:
def getSimilaritiesTotal(dataFrame):
    similarities = []
    for i in tqdm(range(len(dataFrame))):
        for j in range(i+1,len(dataFrame)):
            first_stock = dataFrame.iloc[i,2:]
            second_stock = dataFrame.iloc[j,2:]
            result = 1 - spatial.distance.cosine(first_stock,second_stock)
            similarities.append((stock_symbols[i], stock_symbols[j], result))
    return similarities

In [101]:
def getSimilaritiesYearly(date,dataFrame):
    similarities = []
    elems = len(dataFrame[date])
    start_col = (np.where(dates == date)[0][0]) + 2 # plus 2 is for indexing away from stock symbol & sector
    end_col = start_col + 12
    for i in tqdm(range(elems)):
        for j in range(i+1,elems):
            first_stock = dataFrame.iloc[i,start_col:end_col]
            second_stock = dataFrame.iloc[j,start_col:end_col]
            result = 1-spatial.distance.cosine(first_stock,second_stock)
            similarities.append((stock_symbols[i], stock_symbols[j], result))
    return similarities

In [102]:
def getAllYears(dataFrame):
    i = 0
    arr = []
    start_date = dates[i]
    curr_year = start_date.year
    while(curr_year < 2021):
        yearly_similarity = getSimilaritiesYearly(start_date,dataFrame)
        arr.append((start_date,yearly_similarity))
        curr_year+=1
        i+=12
        start_date = dates[i]
    return arr

### Getting the total and Yearly similarities

In [38]:
total_similarities = getSimilaritiesTotal(df_total)

  0%|          | 0/2935 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


In [108]:
yearly_similarites = getAllYears(df_total)

  0%|          | 0/2935 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


  0%|          | 0/2935 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


  0%|          | 0/2935 [00:00<?, ?it/s]

...

  0%|          | 0/2935 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

### Exporting
At this point all the stock symbols are exported to csv files, each year has it's own specific set of links. And there is also a csv that contains the total similarity across the entire dataset

In [104]:
def writeStocks():
    with open('data/stocks.csv','w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID	label"])
        for stock in stock_symbols:
            writer.writerow([str(stock_symbols.index(stock))+"\t"+stock])

In [105]:
def writeTotal(threshold):
    with open('data/links_total.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Source	Target	Weight"])
        for link in total_similarities:
            first_stock_index = str(stock_symbols.index(link[0]))
            second_stock_index = str(stock_symbols.index(link[1]))
            link_weight = link[2]
            if(abs(link_weight) > threshold):
                writer.writerow([first_stock_index+"\t"+second_stock_index+"\t"+str(link_weight)])

In [123]:
def writeYearly(yearly_calc,threshold):
    for year in yearly_calc:
        curr_year = year[0].year
        links = year[1]
        file_name = 'data/links_'+str(curr_year)+'.csv'
        with open(file_name, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Source	Target	Weight"])
            for link in links:
                first_stock_index = str(stock_symbols.index(link[0]))
                second_stock_index = str(stock_symbols.index(link[1]))
                link_weight = link[2]
                if(abs(link_weight) > threshold):
                    writer.writerow([first_stock_index+"\t"+second_stock_index+"\t"+str(link_weight)])

In [114]:
writeStocks()
writeTotal(0.5)

In [125]:
writeYearly(yearly_similarites,0.6)

### Visualizing the Data
Now that the data has been processed and exported into their own csv's , we can begin to do some visualiation and more graph theory related calculations.

In [20]:
def filterYearly():
    for i in range(2010,2021):
        df_link=pd.read_csv("data/links_"+str(i)+".csv",delimiter="\t")
        df_link = df_link.sort_values(by=['Weight'],key=abs,ascending=False,ignore_index=True)
        #print(df_link[:100000])
        df_link[:100000].to_csv("data/Filtered/links_"+str(i)+".csv",sep='\t')


In [21]:
def filterTotal():
        df_link=pd.read_csv("data/links_total.csv",delimiter="\t")
        df_link = df_link.sort_values(by=['Weight'],key=abs,ascending=False,ignore_index=True)
        #print(df_link[:100000])
        df_link[:100000].to_csv("data/Filtered/links_total.csv",sep="\t")


In [22]:
filterYearly()
filterTotal()

       Source  Target    Weight
0         604    1361  1.000000
1        1170    1171  1.000000
2        1318    1767  1.000000
3         130     140  0.998890
4        2215    2216  0.995275
...       ...     ...       ...
99995    2243    2685  0.788682
99996       7    2565  0.788681
99997     139     942  0.788681
99998    1097    2912  0.788680
99999    1730    1835  0.788679

[100000 rows x 3 columns]
       Source  Target    Weight
0        1170    1171  1.000000
1        1318    1767  1.000000
2         604    1361  1.000000
3         130     140  0.996285
4        2307    2308  0.995681
...       ...     ...       ...
99995     494    1832  0.811056
99996     414    1733  0.811055
99997    1201    2800  0.811055
99998      53     801  0.811053
99999    2543    2687  0.811053

[100000 rows x 3 columns]
       Source  Target    Weight
0        1587    2859  1.000000
1        1170    2859  1.000000
2        1851    2859  1.000000
3        2145    2859  1.000000
4        2470    2